In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [5]:

use_cuda = torch.cuda.is_available()
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.convolutional_layer = nn.Sequential(
            nn.Conv2d(in_channels = 1, out_channels = 4, kernel_size = 5), # 28*28*1 -> 24*24*4
            nn.ReLU(),
            nn.BatchNorm2d(4), # 4 for 4 channels
            #nn.Dropout(0.25),

            nn.Conv2d(in_channels = 4, out_channels = 8, kernel_size = 3), # 24*24*4 -> 22*22*8
            nn.ReLU(),
            nn.BatchNorm2d(8), # 8 for 8 channels
            nn.MaxPool2d(kernel_size = 2, stride = 2), # 22*22*8 - 11*11*8
            nn.Dropout(0.2),

            nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 3), # 11*11*8 -> 9*9*16
            nn.ReLU(),
            nn.BatchNorm2d(16), # 16 for 16 channels
            #nn.MaxPool2d(kernel_size = 2, stride = 2), # 20*20*8 - 10*10*8
            nn.Dropout(0.2),

            nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3), # 9*9*16 -> 7*7*32
            nn.ReLU(),
            nn.BatchNorm2d(32), # 32 for 32 channels
            #nn.Dropout(0.2),

            nn.Conv2d(in_channels = 32, out_channels = 16, kernel_size = 3), # 7*7*32 -> 5*5*16
            nn.ReLU(),
            nn.BatchNorm2d(16), # 16 for 16 channels
            nn.Dropout(0.2),

            nn.AvgPool2d(5)
        )

        self.fc_layer = nn.Sequential(
            nn.Linear(1*1*16,10)

        )

    def forward(self, x):
        x = self.convolutional_layer(x)
        #print(x.shape)
        x = x.reshape(-1, 1*1*16)
        #print(x.shape)
        x = self.fc_layer(x)
        x = F.log_softmax(x, dim=1)
        return x

In [7]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 4, 24, 24]             104
              ReLU-2            [-1, 4, 24, 24]               0
       BatchNorm2d-3            [-1, 4, 24, 24]               8
            Conv2d-4            [-1, 8, 22, 22]             296
              ReLU-5            [-1, 8, 22, 22]               0
       BatchNorm2d-6            [-1, 8, 22, 22]              16
         MaxPool2d-7            [-1, 8, 11, 11]               0
           Dropout-8            [-1, 8, 11, 11]               0
            Conv2d-9             [-1, 16, 9, 9]           1,168
             ReLU-10             [-1, 16, 9, 9]               0
      BatchNorm2d-11             [-1, 16, 9, 9]              32
          Dropout-12             [-1, 16, 9, 9]               0
           Conv2d-13             [-1, 32, 7, 7]           4,640
             ReLU-14             [-1, 3

In [8]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [10]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    print('Epoch : {}'.format(epoch))
    test(model, device, test_loader)

loss=0.14676840603351593 batch_id=468: 100%|██████████| 469/469 [00:40<00:00, 11.51it/s]


Epoch : 1

Test set: Average loss: 0.0805, Accuracy: 9777/10000 (97.77%)



loss=0.07971180975437164 batch_id=468: 100%|██████████| 469/469 [00:39<00:00, 11.77it/s]


Epoch : 2

Test set: Average loss: 0.0499, Accuracy: 9851/10000 (98.51%)



loss=0.05716443434357643 batch_id=468: 100%|██████████| 469/469 [00:39<00:00, 11.90it/s]


...

Epoch : 18

Test set: Average loss: 0.0201, Accuracy: 9934/10000 (99.34%)



loss=0.030504658818244934 batch_id=468: 100%|██████████| 469/469 [00:40<00:00, 11.59it/s]


Epoch : 19

Test set: Average loss: 0.0206, Accuracy: 9937/10000 (99.37%)



loss=0.05704639479517937 batch_id=468: 100%|██████████| 469/469 [00:40<00:00, 11.59it/s]


Epoch : 20

Test set: Average loss: 0.0242, Accuracy: 9916/10000 (99.16%)

